In [ ]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.stats import mode
from scipy.integrate import cumulative_trapezoid
from scipy.signal import correlate

import gc # garbage collector for removing large variables from memory instantly 
import importlib #for force updating changed packages 

#import harp
import harp_resources.process
import harp_resources.utils
from harp_resources import process, utils # Reassign to maintain direct references for force updating 
#from sleap import load_and_process as lp

In [ ]:
common_resampled_rate = 1000 #in Hz

#Cohort 1 visual mismatch THIS
data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/Visual_mismatch_day3/B6J2717-2024-12-10T12-17-03')
save_path = data_path.parent / f"{data_path.name}_processedData"
session_name = "_".join(data_path.parts[-2:])

# Load the saved parquet files
photometry_tracking_encoder_data = pd.read_parquet(save_path / "photometry_tracking_encoder_data_downsampled.parquet", engine="pyarrow")
camera_photodiode_data = pd.read_parquet(save_path / "camera_photodiode_data.parquet", engine="pyarrow")
print("✅ Loaded the saved parquet files")


In [ ]:
process.plot_figure_1(photometry_tracking_encoder_data, session_name, save_path, common_resampled_rate, show_figure = False, downsample_factor=50)

In [ ]:
# Filter the data for the specified time range
filtered_data = camera_photodiode_data.between_time('01:59:16.96', '01:59:17')

# Plot the Photodiode column
plt.figure(figsize=(15, 5))
plt.plot(filtered_data.index, filtered_data['Photodiode'], label='Photodiode')
plt.xlabel('Time')
plt.ylabel('Photodiode')
plt.title('Photodiode Transitions Over Time')
plt.show()